In [1]:
import sys
sys.path.append("../../")  # access to local modules

In [2]:
import os
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import pandas as pd

from tensorflow.train import (
    BytesList,
    FloatList,
    Int64List,
    Feature,
    Features,
    Example
)

2025-04-02 07:41:59.681609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from MLDemo import DataPulling

In [4]:
HOUSING_DIRECTORY = ".tmp/housing"
N_READERS: int = 5

In [5]:
x = tf.range(10)
dataset: tf.data.Dataset = tf.data.Dataset.from_tensor_slices(x)
dataset

2025-04-02 07:42:01.948906: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-02 07:42:03.817791: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-02 07:42:03.817837: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-02 07:42:03.817847: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2397] Ignoring visible gpu device (device: 0, name: AMD Radeon Graphics, pci bus id: 0000:65:00.0) with AMDGPU version : gfx1103. The supported AMDGPU versions are gfx900, gfx906, gfx908, gfx90a, gfx940, gfx941, gfx942, gfx1030, gfx1100,

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [6]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


2025-04-02 07:42:03.840271: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
x_nested = {"a": ([1, 2, 3], [4, 5, 6]), "b": [7, 8, 9]}
dataset = tf.data.Dataset.from_tensor_slices(x_nested)
for item in dataset:
    print(item)

{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


2025-04-02 07:42:03.852094: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [9]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3).batch(7, drop_remainder=True)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)


2025-04-02 07:42:03.878057: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset = dataset.repeat(3).batch(7).map(lambda x: x * 2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [11]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3).batch(7).map(lambda x: x * 2, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.filter(lambda x: tf.reduce_sum(x) > 50)  # do only one lambda per line!
for item in dataset:
    print(item)

tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)


In [12]:
dataset = tf.data.Dataset.range(10).repeat(2)
dataset = dataset.shuffle(buffer_size=4, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 4 2 3 5 0 6], shape=(7,), dtype=int64)
tf.Tensor([9 8 2 0 3 1 4], shape=(7,), dtype=int64)
tf.Tensor([5 7 9 6 7 8], shape=(6,), dtype=int64)


In [13]:
train_filepaths = DataPulling.to_randomized_files(DataPulling.open_tgz(DataPulling.HOUSING)[0][1], HOUSING_DIRECTORY, number_of_files=50)
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [14]:
dataset = filepath_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=N_READERS)

In [15]:
for item in dataset.take(5):
    print(item)

tf.Tensor(b'3298,-118.15,33.91,35.0,1590.0,350.0,1299.0,335.0,4.0313,163200.0,<1H OCEAN', shape=(), dtype=string)
tf.Tensor(b'4916,-119.69,36.81,13.0,1524.0,366.0,994.0,370.0,2.5446,93800.0,INLAND', shape=(), dtype=string)
tf.Tensor(b'3386,-119.19,35.41,12.0,2835.0,471.0,1399.0,413.0,4.4125,149000.0,INLAND', shape=(), dtype=string)
tf.Tensor(b'15543,-118.33,34.02,46.0,2223.0,361.0,968.0,373.0,4.26,182600.0,<1H OCEAN', shape=(), dtype=string)
tf.Tensor(b'10746,-117.06,32.56,17.0,2803.0,683.0,2768.0,676.0,1.7958,140400.0,NEAR OCEAN', shape=(), dtype=string)


2025-04-02 07:42:04.087311: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
def parse_csv_line(line, n_inputs=8):
    defs = [tf.constant([], dtype=tf.float32)] + [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[1:-2]), tf.stack(fields[-2:-1])

In [17]:
def preprocess(line):
    x, y = parse_csv_line(line)
    # todo normalization
    return x, y

In [18]:
def csv_reader_dataset(filepaths, n_readers=5, nread_threads=None, n_parse_threads=5, shuffle_buffer_size=10000, seed=42, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths, seed=seed)
    dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers, num_parallel_calls=n_parse_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    return dataset.batch(batch_size).prefetch(1)

In [19]:
train_set = csv_reader_dataset(train_filepaths[:30])
valid_set = csv_reader_dataset(train_filepaths[30:40])
test_set = csv_reader_dataset(train_filepaths[40:])

In [20]:
with tf.io.TFRecordWriter(".tmp/my_data.tfrecord") as f:
    f.write(b"This is the first line")
    f.write(b"This is the second line")

In [21]:
dataset = tf.data.TFRecordDataset([".tmp/my_data.tfrecord"])
for item in dataset:
    print(item)
    print(item.numpy())
    print(item.numpy().decode('utf-8'))

tf.Tensor(b'This is the first line', shape=(), dtype=string)
b'This is the first line'
This is the first line
tf.Tensor(b'This is the second line', shape=(), dtype=string)
b'This is the second line'
This is the second line


In [22]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter(".tmp/my__compressed_data.tfrecord", options) as f:
    f.write(b"This is a compressed line")

In [23]:
dataset = tf.data.TFRecordDataset([".tmp/my__compressed_data.tfrecord"], compression_type="GZIP")
for item in dataset:
    print(item)
    print(item.numpy())
    print(item.numpy().decode('utf-8'))

tf.Tensor(b'This is a compressed line', shape=(), dtype=string)
b'This is a compressed line'
This is a compressed line


In [24]:
person_example = Example(
    features=Features(
        feature = {
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }
    )
)

In [25]:
with tf.io.TFRecordWriter(".tmp/my_contacts.tfrecord") as f:
    for _ in range(5):
        f.write(person_example.SerializeToString())

In [26]:
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}

def parse(serialized_examples):
    # return tf.io.parse_single_example(serialized_examples, feature_description)
    return tf.io.parse_example(serialized_examples, feature_description)


In [27]:
dataset = tf.data.TFRecordDataset([".tmp/my_contacts.tfrecord"]).map(parse)
for parsed_example in dataset:
    print(parsed_example)

{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
{'emails': SparseTensor(indices=tf.Tenso

# Normalization Layers in Keras

In [28]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [29]:
# including in the model with training
norm_layer = tf.keras.layers.Normalization()
model = tf.keras.models.Sequential([
    norm_layer,
    tf.keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=2e-3))
norm_layer.adapt(X_train)  # this calculates the mean and standard deviation of X_train
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=5)

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - loss: 4.5333 - val_loss: 2.2754
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - loss: 0.8457 - val_loss: 0.6168
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 0.6299 - val_loss: 0.8097
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.5914 - val_loss: 0.9616
Epoch 5/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 0.5951 - val_loss: 0.6150


In [30]:
# not in training phase for speedup
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train)
X_train_scaled = norm_layer(X_train)
X_valid_scaled = norm_layer(X_valid)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=2e-3))
model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), epochs=5)

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 3.8299 - val_loss: 1.3675
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 0.8667 - val_loss: 0.6552
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - loss: 0.6846 - val_loss: 1.4635
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - loss: 0.6466 - val_loss: 0.6793
Epoch 5/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 0.6458 - val_loss: 0.9527


In [31]:
# FUUUUUSIOOOOON!
final_model = tf.keras.Sequential([norm_layer, model])

## Discretization

In [32]:
age = tf.constant([[10.], [93.], [57.], [18.], [37.], [5.]])

In [33]:
discretize_layer = tf.keras.layers.Discretization(bin_boundaries=[18., 50.])
age_categories = discretize_layer(age)
age_categories

<tf.Tensor: shape=(6, 1), dtype=int64, numpy=
array([[0],
       [2],
       [2],
       [1],
       [1],
       [0]])>

In [34]:
discretize_layer = tf.keras.layers.Discretization(num_bins=3)
discretize_layer.adapt(age)
age_categories = discretize_layer(age)
age_categories

<tf.Tensor: shape=(6, 1), dtype=int64, numpy=
array([[1],
       [2],
       [2],
       [1],
       [2],
       [0]])>

In [35]:
# one-hot encoder
onehot_layer = tf.keras.layers.CategoryEncoding(num_tokens=3)
onehot_layer(age_categories)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)>

In [36]:
two_age_categories = np.array([[1, 0], [2, 2], [2, 0]])
onehot_layer(two_age_categories)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 1., 0.],
       [0., 0., 1.],
       [1., 0., 1.]], dtype=float32)>

In [37]:
onehot_layer = tf.keras.layers.CategoryEncoding(num_tokens=3 + 3)
onehot_layer(two_age_categories + [0, 3])  # shift by three

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.]], dtype=float32)>

In [38]:
cities = ["Auckland", "Paris", "Paris", "San Francisco"]
str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Auckland"], ["Montreal"]])

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[1],
       [3],
       [3],
       [0]])>

In [39]:
str_lookup_layer = tf.keras.layers.StringLookup(output_mode="one_hot")
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Auckland"], ["Montreal"]])

<tf.Tensor: shape=(4, 4), dtype=int64, numpy=
array([[0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])>

In [40]:
str_lookup_layer = tf.keras.layers.StringLookup(num_oov_indices=5)  # oov = out of vocabulary
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Foo"], ["Bar"], ["Baz"]])

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[5],
       [7],
       [4],
       [3],
       [4]])>

In [41]:
hashing_layer = tf.keras.layers.Hashing(num_bins=10)  # do not use if possible since this can still create collisions
hashing_layer([["Paris"], ["Tokyo"], ["Auckland"], ["Montreal"]])

<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[0],
       [1],
       [9],
       [1]])>

In [42]:
# embedding

tf.random.set_seed(42)
embedding_layer = tf.keras.layers.Embedding(input_dim=5, output_dim=2)
embedding_layer(np.array([2, 4, 2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.04668725,  0.04036435],
       [ 0.0357411 , -0.02600536],
       [-0.04668725,  0.04036435]], dtype=float32)>

In [43]:
# string embedding
tf.random.set_seed(42)
ocean_prox = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(ocean_prox)
lookup_and_embed = tf.keras.models.Sequential([str_lookup_layer, tf.keras.layers.Embedding(input_dim=str_lookup_layer.vocabulary_size(), output_dim=2)])
lookup_and_embed(np.array(["<1H OCEAN", "ISLAND", "<1H OCEAN"]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.04587116, -0.02930641],
       [ 0.03589033, -0.04210738],
       [ 0.04587116, -0.02930641]], dtype=float32)>

## Preprocessing

In [44]:
train_data = ["To be", "!(to be)", "That's the question", "be, be, be."]
text_vec_layer = tf.keras.layers.TextVectorization()
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])

<tf.Tensor: shape=(2, 4), dtype=int64, numpy=
array([[2, 1, 0, 0],
       [6, 2, 1, 2]])>

In [45]:
text_vec_layer = tf.keras.layers.TextVectorization(output_mode="tf_idf")  # term-frequency x inverse-document-frequency
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.96725637, 0.6931472 , 0.        , 0.        , 0.        ,
        0.        ],
       [0.96725637, 1.3862944 , 0.        , 0.        , 0.        ,
        1.0986123 ]], dtype=float32)>

## TensorFlow Hub

In [46]:
#  geht gerade nicht... import tensorflow_hub as hub

## Image Preprocessing Layers

In [47]:
from sklearn.datasets import load_sample_images

images = load_sample_images()["images"]
crop_image_layer = tf.keras.layers.CenterCrop(height=100, width=100)
cropped_images = crop_image_layer(images)

## Datasets

In [48]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]

In [49]:
for batch in mnist_train.shuffle(10000, seed=42).batch(32).prefetch(1):
    images = batch["image"]
    labels = batch["label"]
    ...

In [50]:
mnist_train = mnist_train.shuffle(buffer_size=10000, seed=42).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))
mnist_train = mnist_train.prefetch(1)